In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report


In [ ]:
# Carregar dataset (substituir pelo seu dataset se necessário)
dataset = load_dataset("??")  # Insira o nome correto da base em português

# Divisão entre treino (70%), validação (10%) e teste (20%)
train_test_split = dataset["train"].train_test_split(test_size=0.3)
test_valid_split = train_test_split["test"].train_test_split(test_size=2/3)

dataset_final = DatasetDict({
    "train": train_test_split["train"],
    "validation": test_valid_split["train"],
    "test": test_valid_split["test"]
})

# Exibir número de exemplos por divisão
for split in dataset_final:
    print(f"{split}: {len(dataset_final[split])} exemplos")


In [ ]:
# Carregar o tokenizer do BERTimbau
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

# Função para tokenizar
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Aplicar tokenização
tokenized_datasets = dataset_final.map(tokenize_function, batched=True)

# Visualizar tokens de um exemplo
example_text = dataset_final["train"][0]["text"]
tokens = tokenizer.tokenize(example_text)
print("Tokens de um documento:")
print(tokens)


In [ ]:
# Carregar modelo BERTimbau pré-treinado para classificação
model = AutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=2)

# Configurar os parâmetros de treinamento
training_args = TrainingArguments(
    output_dir="./results_bertimbau",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs_bertimbau"
)

# Função de avaliação com métricas
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    f1_micro = f1_score(labels, preds, average="micro")
    f1_macro = f1_score(labels, preds, average="macro")
    return {"accuracy": acc, "f1_micro": f1_micro, "f1_macro": f1_macro}

# Criar Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Iniciar o treinamento
trainer.train()


In [ ]:
# Fazer previsões
predictions = trainer.predict(tokenized_datasets["test"])
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

# Calcular métricas principais
acc = accuracy_score(labels, preds)
f1_micro = f1_score(labels, preds, average="micro")
f1_macro = f1_score(labels, preds, average="macro")
conf_matrix = confusion_matrix(labels, preds)

# Exibir métricas principais
print(f"Acurácia: {acc:.4f}")
print(f"F1-score (Micro): {f1_micro:.4f}")
print(f"F1-score (Macro): {f1_macro:.4f}")

# Exibir matriz de confusão
plt.figure(figsize=(6,6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predito")
plt.ylabel("Real")
plt.title("Matriz de Confusão")
plt.show()

# **Relatório detalhado das métricas**
print("Relatório de Classificação:\n")
print(classification_report(labels, preds, target_names=["Negativo", "Positivo"]))
